<a href="https://colab.research.google.com/github/mrjohnokolie/dem-enhancement/blob/main/initial_xgb_cult_cop_Github_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from numpy.random import seed
seed(42)

In [4]:


# Import training_test data (Site 1)
from google.colab import files
uploaded = files.upload()

Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_clean_t41_v43_C_merge.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_clean_t41_v43_C_merge.csv


In [5]:
# Install packages
!pip install xgboost
!pip install catboost
!pip install lightgbm
!pip install --upgrade ngboost
!pip install ada-boost
!pip install -U scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=f189e3d92fdbbf8c89d2708112647591c12fcc2037af35011b7ce2099b178307
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 34.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [6]:
# Import libraries

import pandas  as pd
import numpy   as np

import lightgbm as lgb
import catboost as cgb
import xgboost as xgb

import sklearn
from sklearn.model_selection import train_test_split
from pandas import read_csv
from matplotlib import pyplot
from numpy import absolute
from numpy import mean
from numpy import std

from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import joblib
from matplotlib import pyplot as plt

# Accuracy metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

# Additional
import math
from scipy.stats import uniform
import time #2
from sklearn.inspection import permutation_importance
from xgboost import plot_importance

# Regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from ngboost import NGBRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import seaborn as sns # for correlation heatmap


In [7]:
# Read training/test/val data

data = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_clean_t41_v43_C_merge.csv")
data = data.reset_index()
X,y = data.iloc[:,7:15], data.iloc[:,6]
y

0         0.139503
1        -0.169899
2        -0.324997
3        -0.158798
4         0.397400
           ...    
46172    12.073002
46173     9.778599
46174    12.559099
46175     4.479000
46176    12.201498
Name: Elevation Diff, ∆h (m), Length: 46177, dtype: float64

In [8]:
# Split training and test data,  Train - 80%, test - 20%

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)
print(X_train.shape), print(y_train.shape)
print(X_val.shape), print(y_val.shape)

(36941, 8)
(36941,)
(9236, 8)
(9236,)


(None, None)

**MODEL FIT INITIAL - PERFORMANCE, LEARNING CURVES, EARLY STOPPING**

In [10]:
# Model fitting - default/initial

start_time = time.time()
xgb_model = XGBRegressor().fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

# early_stopping_rounds=10, eval_metric=["mae","rmse"], eval_set=eval_set, verbose=True

--- 2.544604778289795 seconds ---


**MODEL PREDICTIONS - INITIAL**

In [ ]:
# Prediction on test/val set - default/initial
start_time = time.time()
y_pred_ts1 = xgb_model.predict(X_val)
print("--- %s seconds ---" % (time.time() - start_time))

In [12]:
# Prediction on training set - default/initial

start_time = time.time()
y_pred_tr1 = xgb_model.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.07611441612243652 seconds ---


**EXPORT Y_VAL AND Y_PRED FOR PLOTTING**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# convert array into dataframe
DFpred = pd.DataFrame(y_pred_ts1)
DFval = pd.DataFrame(y_val)

# save the dataframe as a csv file
DFpred.to_csv("/content/drive/My Drive/GITHUB_init_xgb_cop_cult_y_pred_ts1.csv", index = False, header=True)
DFval.to_csv("/content/drive/My Drive/GITHUB_init_xgb_cop_cult_y_val.csv", index = False, header=True)

# read saved csvs
DFpred_csv = pd.read_csv("/content/drive/My Drive/GITHUB_init_xgb_cop_cult_y_pred_ts1.csv")
DFval_csv = pd.read_csv("/content/drive/My Drive/GITHUB_init_xgb_cop_cult_y_val.csv")

# Merge files
DFmerge_csv = pd.concat([DFpred_csv, DFval_csv], axis=1)

# Save to csv
DFmerge_csv.to_csv("/content/drive/My Drive/GITHUB_init_xgb_cop_cult_y_pred_y_val_merge.csv", index = False, header=True)

**PREDICTION ERROR - INITIAL**

In [ ]:
# Test Error -  default/initial
test_mae1 = mean_absolute_error(y_val, y_pred_ts1)
test_mdae1 = median_absolute_error(y_val, y_pred_ts1) #robust to outliers
test_mse1 = mean_squared_error(y_val, y_pred_ts1)
test_rmse1 = mean_squared_error(y_val, y_pred_ts1, squared=False)
test_maxe1 = max_error(y_val, y_pred_ts1)
test_mape1 = mean_absolute_percentage_error(y_val, y_pred_ts1)
test_R21 = r2_score(y_val, y_pred_ts1)

In [ ]:
# Print Test Error - default/initial

print("\n")
print("Test error - initial")
print("test_mae:")
print(test_mae1)
print("test_mdae:")
print(test_mdae1)
print("test_mse:")
print(test_mse1)
print("test_rmse:")
print(test_rmse1)
print("test_maxe:")
print(test_maxe1)
print("test_mape:")
print(test_mape1)
print("test_R2:")
print(test_R21)

In [ ]:
# Train Error - initial

train_mae1 = mean_absolute_error(y_train, y_pred_tr1)
train_mdae1 = median_absolute_error(y_train, y_pred_tr1) #robust to outliers
train_mse1 = mean_squared_error(y_train, y_pred_tr1)
train_rmse1 = mean_squared_error(y_train, y_pred_tr1, squared=False)
train_maxe1 = max_error(y_train, y_pred_tr1)
train_mape1 = mean_absolute_percentage_error(y_train, y_pred_tr1)
train_R21 = r2_score(y_train, y_pred_tr1)

In [ ]:
# Print Train Error - initial
print("\n")
print("Train error - initial")
print("train_mae:")
print(train_mae1)
print("train_mdae:")
print(train_mdae1)
print("train_mse:")
print(train_mse1)
print("train_rmse:")
print(train_rmse1)
print("train_maxe:")
print(train_maxe1)
print("trrain_mape:")
print(train_mape1)
print("train_R2:")
print(train_R21)

**MODEL EXPLAINABILITY - PARTIAL DEPENDENCE PLOTS (PDP) AND SHAP VALUES PDP -** https://pdpbox.readthedocs.io/en/latest/pdp_plot.html

In [ ]:
features = ['Elevation (m)','Slope (°)','Aspect (°)','Surface Roughness','Topographic Position Index','Terrain Ruggedness Index','Terrain Surface Texture','Vector Ruggedness Measure']


In [ ]:
# Feature importance - default/initial
xgb_model.feature_importances_

In [ ]:
# Plot feature importance - default/initial
plt.barh(features, xgb_model.feature_importances_)

In [ ]:
# Plot F_score - default/initial
plot_importance(xgb_model)
plt.show()

**SAVE MODEL**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# save model with joblib - intial
joblib.dump(xgb_model, "/content/drive/My Drive/GITHUB_default_xgb_cop_41_43.joblib")

**SITE B VALIDATION - INITIAL**

In [ ]:
# Load models

default_model = joblib.load("/content/drive/My Drive/GITHUB_default_xgb_cop_41_43.joblib")

[17:12:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# Import independent validation data - all sites
from google.colab import files
uploaded = files.upload()

Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_42.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_42.csv
Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_45.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_45.csv
Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_46.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_46.csv
Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_47.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_47.csv
Saving LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_48.csv to LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_48.csv


In [ ]:
# specify predictor variables 42
data42 = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_42.csv")
data42 = data42.reset_index()
X42,y42 = data42.iloc[:,7:15], data.iloc[:,6]
X42

,Elevation (m),Slope (°),Aspect (°),Surface Roughness,Topographic Position Index,Terrain Ruggedness Index,Terrain Surface Texture,Vector Ruggedness Measure
0,132.9300,2.27006,0.003311,2.56882,0.035263,2.84954,0.226378,0.000019
1,291.6890,16.22810,0.006004,19.46120,1.885620,21.74080,30.487200,0.001672
2,70.3140,4.06604,0.006767,4.52854,0.142006,5.26427,3.739870,0.000031
3,228.5000,9.17943,0.012627,15.33150,-2.908170,15.86820,31.101200,0.008856
4,111.3550,3.97560,0.013313,4.39862,0.046196,5.05020,1.303700,0.000048
...,...,...,...,...,...,...,...,...
134688,65.1505,3.70693,359.989000,4.26924,0.063347,4.72448,1.955260,0.000037
134689,109.3890,3.01079,359.990000,3.55148,0.126923,3.98111,3.045820,0.000013
134690,136.3010,2.17084,359.990000,2.58942,0.324005,2.90553,0.906688,0.000074
134691,88.2406,1.05919,359.993000,1.37218,0.371040,1.83053,0.980032,0.000101


In [ ]:
# specify predictor variables 45
data45 = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_45.csv")
data45 = data45.reset_index()
X45,y45 = data45.iloc[:,7:15], data.iloc[:,6]
X45

,Elevation (m),Slope (°),Aspect (°),Surface Roughness,Topographic Position Index,Terrain Ruggedness Index,Terrain Surface Texture,Vector Ruggedness Measure
0,103.9960,0.453065,195.7410,1.047580,-0.080650,0.960945,1.282610,0.000073
1,103.8440,0.552654,148.9600,0.752586,0.052429,0.749441,1.115240,0.000006
2,103.3850,0.533275,137.3810,0.827415,-0.281715,1.070830,0.968426,0.000015
3,103.4450,0.277632,86.8515,0.446037,-0.054588,0.455196,0.839689,0.000013
4,103.5700,0.178504,161.9240,0.588783,0.178185,0.668764,0.727287,0.000017
...,...,...,...,...,...,...,...,...
23104,68.7834,2.140730,113.9150,3.106870,0.046303,2.881490,0.500210,0.000059
23105,67.4714,2.366260,127.2090,3.121130,-0.391022,3.218420,0.418520,0.000116
23106,66.6632,1.778120,170.1620,2.449580,-0.705086,3.093320,0.347006,0.000139
23107,67.3502,1.947020,196.6660,2.321720,0.144646,2.528060,0.284882,0.000017


In [ ]:
# specify predictor variables 46
data46 = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_46.csv")
data46 = data46.reset_index()
X46,y46 = data46.iloc[:,7:15], data.iloc[:,6]
y46

0         0.139503
1        -0.169899
2        -0.324997
3        -0.158798
4         0.397400
           ...    
46172    12.073002
46173     9.778599
46174    12.559099
46175     4.479000
46176    12.201498
Name: Elevation Diff, ∆h (m), Length: 46177, dtype: float64

In [ ]:
# specify predictor variables 47
data47 = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_47.csv")
data47 = data47.reset_index()
X47,y47 = data47.iloc[:,7:15], data.iloc[:,6]
X47

,Elevation (m),Slope (°),Aspect (°),Surface Roughness,Topographic Position Index,Terrain Ruggedness Index,Terrain Surface Texture,Vector Ruggedness Measure
0,76.6691,1.90379,292.460,2.44662,-0.087303,2.40250,0.000000,0.000017
1,77.6992,2.04168,289.683,2.54646,-0.016968,2.58818,0.000000,0.000007
2,78.7744,1.91523,288.737,2.62361,0.163498,2.52600,0.000000,0.000010
3,79.6084,1.52747,292.972,2.05497,0.125603,1.98260,0.000000,0.000012
4,80.2660,1.44484,296.495,2.10189,0.066925,1.88907,0.000000,0.000007
...,...,...,...,...,...,...,...,...
22361,83.8435,7.45492,192.945,9.80819,-1.425050,10.56960,9.559250,0.000963
22362,84.6755,6.85136,196.435,9.80819,-1.454430,9.91485,9.786090,0.001379
22363,67.0177,2.22333,123.136,2.57902,-0.245445,2.83598,0.018838,0.000120
22364,66.1059,1.25423,137.226,2.24879,-0.534721,2.48880,0.019053,0.000232


In [ ]:
# specify predictor variables 48
data48 = pd.read_csv("LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_48.csv")
data48 = data48.reset_index()
X48,y48 = data48.iloc[:,7:15], data.iloc[:,6]
y48

0         0.139503
1        -0.169899
2        -0.324997
3        -0.158798
4         0.397400
           ...    
46172    12.073002
46173     9.778599
46174    12.559099
46175     4.479000
46176    12.201498
Name: Elevation Diff, ∆h (m), Length: 46177, dtype: float64

In [ ]:
# Make a prediction - initial 42
start_time = time.time()
y_pred_init_42 = default_model.predict(X42)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.25528430938720703 seconds ---


In [ ]:
# Make a prediction - initial 45
start_time = time.time()
y_pred_init_45 = default_model.predict(X45)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04513096809387207 seconds ---


In [ ]:
# Make a prediction - initial 46
start_time = time.time()
y_pred_init_46 = default_model.predict(X46)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.043659210205078125 seconds ---


In [ ]:
# Make a prediction - initial 47
start_time = time.time()
y_pred_init_47 = default_model.predict(X47)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.042532920837402344 seconds ---


In [ ]:
# Make a prediction - initial 48
start_time = time.time()
y_pred_init_48 = default_model.predict(X48)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04111504554748535 seconds ---


In [ ]:
# append new predictions to csv file - initial 42
data42["C_HEM_init"] = y_pred_init_42
data42.to_csv("/content/drive/My Drive/LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_42_init_xgb_pred.csv", index = False)

In [ ]:
# append new predictions to csv file - initial 45
data45["C_HEM_init"] = y_pred_init_45
data45.to_csv("/content/drive/My Drive/LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_45_init_xgb_pred.csv", index = False)

In [ ]:
# append new predictions to csv file - initial 46
data46["C_HEM_init"] = y_pred_init_46
data46.to_csv("/content/drive/My Drive/LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_46_init_xgb_pred.csv", index = False)

In [ ]:
# append new predictions to csv file - initial 47
data47["C_HEM_init"] = y_pred_init_47
data47.to_csv("/content/drive/My Drive/LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_47_init_xgb_pred.csv", index = False)

In [ ]:
# append new predictions to csv file - initial 48
data48["C_HEM_init"] = y_pred_init_48
data48.to_csv("/content/drive/My Drive/LIDAR_30m_cubic_pts_clean_parameters_egm08_41_48_C_48_init_xgb_pred.csv", index = False)